# Processing the Indian Universe File 

In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
from scipy import stats

In [2]:
# Helper Functions
def coalesce(x, y=0):
    return np.where(pd.isna(x), y, x)

def select_stocks(data, rank_column, n_stocks=30):
    
    unique_dates = data['date'].unique()
    portfolio_data = pd.DataFrame()

    for month_date in unique_dates:
        month_data = data[data['date'] == month_date]
        ordered_data = month_data.sort_values(by=rank_column)

        selected_stocks = pd.DataFrame()
        sector_count = pd.DataFrame(columns=['Sector', 'Count'])
        mcap_count = pd.DataFrame({'Category': ['Large', 'Mid', 'Small'],
                                   'Count': [0, 0, 0],
                                   'Max_Count': [16, 14, 12]})

        for _, stock in ordered_data.iterrows():
            sector = stock['TRBC.Economic.Sector.Name']
            category = stock['mcap_cat']

            if sector not in sector_count['Sector'].values:
                sector_count = pd.concat([sector_count, pd.DataFrame([[sector, 0]], columns=['Sector', 'Count'])], ignore_index=True)
                #print(sector_count)
                
            if (sector_count.loc[sector_count['Sector'] == sector, 'Count'].values[0] < 12 and
                mcap_count.loc[mcap_count['Category'] == category, 'Count'].values[0] < mcap_count.loc[mcap_count['Category'] == category, 'Max_Count'].values[0]):
                
                selected_stocks = pd.concat([selected_stocks, stock.to_frame().T])
                #print(stock)
                
                sector_count.loc[sector_count['Sector'] == sector, 'Count'] += 1
                mcap_count.loc[mcap_count['Category'] == category, 'Count'] += 1

            if len(selected_stocks) == n_stocks:
                break

        portfolio_data = pd.concat([portfolio_data, selected_stocks])
        

    return portfolio_data

In [3]:
## Load the CSV file

filepath = "C:/Users/Vedant Wanchoo/Desktop/CGS 2020/India Fund/India Proposal/Portfolio Assembly India/scores_liqIBUniverse_India_2025-02-07.csv"
scores = pd.read_csv(filepath)

# Filtering the scores file for pick_toScore == True
scores = scores[scores['pick_toScore'] == True]

# Selecting the required columns
cols_select = ["RIC", "date", "MVC", "TRBC.Economic.Sector.Name", "avgDailyUSDollarVolume", "last_ClosePrice", "score_total_ind"]

## Adding the date column and calculating the score_total_ind
scores['date'] = pd.to_datetime("2025-02-07")
scores['score_total_ind'] = (coalesce(scores['score_momentum']) +
                             coalesce(scores['score_profitability']) +
                             coalesce(scores['score_value_adj']) +
                             coalesce(scores['score_vol']) +
                             coalesce(scores['score_st_combined']) +
                             coalesce(scores['score_liqShock']))

C:\Users\Vedant Wanchoo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (139,164,165,289,309,339,404,432,670,671,672,674,681,685,694,696,698,703,706,708,988) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# Filtering and creating variables to filter based on market cap
scores_filtered = scores[cols_select].copy()
scores_filtered['mcap_check'] = np.where(scores_filtered['MVC'] > scores_filtered['MVC'].quantile(0.60), 1, 0)
scores_filtered['mcap_rank'] = scores_filtered['MVC'].rank(ascending=False)
scores_filtered['mcap_cat'] = np.where(scores_filtered['mcap_rank'] <= 100, 'Large',
                                       np.where(scores_filtered['mcap_rank'] <= 250, 'Mid', 'Small'))

# Creating the liquidity and price limits
scores_filtered['liquidity_min'] = 4000000
scores_filtered['price_min'] = 200000 / 30

# Applying liquidity and price checks
scores_filtered['liquidity_check'] = np.where(scores_filtered['avgDailyUSDollarVolume'] >= scores_filtered['liquidity_min'], 1, 0)
scores_filtered['price_check'] = np.where(scores_filtered['last_ClosePrice'] <= scores_filtered['price_min'], 1, 0)

# Filtering and ranking the data
scores_filtered = scores_filtered[(scores_filtered['mcap_rank'] <= 500) &
                                  (scores_filtered['liquidity_check'] == 1) &
                                  (scores_filtered['price_check'] == 1)]

scores_filtered['mcap_rank_ST'] = scores_filtered.groupby('score_total_ind')['MVC'].rank(ascending=True) / 100
scores_filtered['score_total_new'] = scores_filtered['score_total_ind'] + scores_filtered['mcap_rank_ST']
scores_filtered['rank_total'] = scores_filtered['score_total_new'].rank(ascending=False)


In [5]:
# Portfolio creation
portfolio = select_stocks(data=scores_filtered, rank_column='rank_total')
portfolio['picked'] = True

In [6]:
# Merging with the original scores file
output = pd.merge(scores, portfolio[['RIC', 'picked']], on='RIC', how='left')

In [7]:
output.head()

,RIC,Market_adj,ISIN,Currency.abb,score_liqShock,score_liqShock_detailed,Market,Full.Name,Business.Description,Country.of.Headquarters,...,score_short_for_percentile,score_short_max_market,score_short_percentile,score_short_percentile_adj,score_total_max_industry,score_total_percentile_industry,score_total_percentile_industry_adj,score_total_hypothetical_percentile,score_total_ind,picked
0,63MO.NS,India,INE111B01023,INR,5.0,86.0,India,63 Moons Technologies,63 Moons Technologies Limited is an India-base...,India,...,-14.0,100.0,26.0,26.0,16.0,14.0,87.500000,96.0,30.0,NaN
1,AARP.NS,India,INE0LRU01027,INR,5.0,83.0,India,Aarti Pharmalabs,Aarti Pharmalabs Limited is a manufacturer of ...,India,...,-12.0,100.0,14.0,14.0,56.0,52.0,92.857143,100.0,39.0,NaN
2,ABB.NS,India,INE117A01022,INR,0.0,67.0,India,ABB India,ABB India Limited is an India-based technology...,India,...,-17.0,100.0,54.0,54.0,37.0,18.0,48.648649,84.0,28.0,NaN
3,ABDL.NS,India,INE442H01029,INR,0.0,76.0,India,Ashoka Buildcon,Ashoka Buildcon Limited is an India-based high...,India,...,-14.0,100.0,26.0,26.0,43.0,30.0,69.767442,96.0,31.0,NaN
4,ABOT.NS,India,INE358A01014,INR,0.0,NaN,India,Abbott India,Abbott India Limited is an India-based company...,India,...,-10.0,100.0,4.0,4.0,56.0,48.0,85.714286,100.0,40.0,NaN


In [8]:
##

# Writing the output to a CSV file
output.to_csv("C:/Users/Vedant Wanchoo/Desktop/CGS 2020/India Fund/India Proposal/Portfolio Assembly India/scores-processed_smallcase_2025-02-07.csv", index=False)

In [9]:
output.shape

(958, 1028)

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats

# Load required libraries
import pandas as pd
import numpy as np
from datetime import datetime

import math

In [11]:
##

data = output.copy()
#data=pd.read_csv(r'C:\Users\Vedant Wanchoo\Desktop\CGS 2020\India Fund\India Proposal\Portfolio Assembly India\scores-processed_smallcase_2024-12-08.csv',encoding= 'unicode_escape')

#Input Prof Ben's CSV file here

In [12]:
data.shape

(958, 1028)

In [13]:
data = data[data['Market'] == 'India']

In [14]:
#data.loc[data['ticker'] == 'F&N', ['ticker']] = 'F N'

In [15]:
data.shape

(958, 1028)

In [16]:
dd=pd.DataFrame()

In [17]:
#dd['IB Ticker']=data['ib_symbol']
dd['IB Ticker']=data['ticker']
dd['Company Name']=data['Full.Name']
dd['Industry']=data['GICS.Industry.Name']
dd['GICS Sector']=data['GICS.Sector.Name']
dd['Business Description']=data['Business.Description']
#dd['Adj Total Score']=data['score_total_percentile_adj']
#dd['score_total_percentile_industry_adj']=data['score_total_percentile_industry_adj']
#dd['Hypothetical TS Rank']=data['score_total_hypothetical_percentile']

dd['Mkt Cap mn USD']=data['MVC']/(1000000*85) #'MVC_USD'Company.Market.Cap_USD
dd['Stock Price']=data['last_ClosePrice']#last_ClosePrice #P_latest

dd['Market']=data['Market']
dd['Currency']=data['CF.CURR']
dd['fx']=data['fx']


dd['Total Score']=data['score_total_ind']
dd['Total Score']=dd['Total Score'].fillna(0)

dd['Value']=data['score_value_adj']
dd['Momentum']=data['score_momentum']
dd['Quality']=data['score_profitability']
dd['Volatility']=data['score_vol']
dd['Proprietary']= data['score_st_combined'] + data['score_liqShock']

dd['ESG Score']=data['ESG.Score']
dd['ESG Combined Score']=data['ESG.Combined.Score']

#dd['Liquidity']=data['score_liqShock']
#dd['Investment']=data['score_investment']
#dd['Ivol (CAPM)']=data['ivol_CAPM']
#dd['Adjusted Value']=data['score_value_adj']
#dd['Liquidity Score']=data['score_liqShock_detailed']
#dd['Short Score']=data['score_short']
#dd['Adj Short Score']=data['score_short_percentile_adj']

#dd['Bombora Boost Score']=data['score_boost_bombora']

dd['P / S']=data['P_S']
dd['P / BV']=data['P_BV']
dd['P / E']=data['P_E']
dd['P / CF']=data['P_CF']
dd['ROE']=data['ROE']/100
dd['ROA']=data['ROA']/100
dd['DE']=data['DE']/100
dd['DA']=data['DA']/100
dd['Dividend Yield']=data['DividendYield']/100
dd['FCF per Share']=data['FCF_perShare']


dd['Islam Flag']=data['Shariah.Compliant.Flag']
dd['kill size']=data['kill_size']
dd['kill liquidity']=data['kill_liquidity']
dd['kill extr Value1']=data['kill_extremeValue1']
dd['kill extr Value2']=data['kill_extremeValue2']


dd['RIC']=data['RIC']
dd['ISIN']=data['ISIN.Code']
dd['Ticker']=data['ticker']
dd['Ticker Symbol']=data['Ticker.Symbol']
dd['Exchange Ticker']=data['Exchange.Ticker']

dd['Country of HQ']=data['Country.of.Headquarters']
dd['Market adj']=data['Market_adj']
dd['Sector Name']=data['NAICS.Sector.Name']
dd['Subsector Name']=data['NAICS.Subsector.Name']
dd['Sub Industry']=data['NAICS.International.Industry.Name']

dd['Dividend Ex Date']=data['Dividend.Ex.Date_Live']
#dd['Dividend Yield (Est.)']=data['approx_div_yield_Live'] #lastmonth_Dividend.yield

dd['Avg_Volume_Daily_USD']=data['avgDailyUSDollarVolume']/1000000

#dd['20 Day Avg Volume (Mn USD)']=data['Average.Daily.Value.Traded...20.Days_USD']

dd.insert(0,'Tick | Curr',dd['IB Ticker'] + " | " + dd['Currency'])

dd['Ideal Portfolio']=data['picked']
dd['Ideal Portfolio'].fillna(False, inplace=True)
dd.loc[dd['Ideal Portfolio'], 'Ideal Portfolio'] = 1/30
dd['Ideal Portfolio'].replace(False, np.nan, inplace=True)

#dd['Dividend Yield1']=data['Dividend.yield']
#dd['Mkt Cap mn USD']=data['Company.Market.Cap_USD']#'Company.Market.Cap_USD'
#dd['Stock Price']=data['P_latest']
#dd['ACWI weight']=data['weight_msci_acwi']
#dd['score total esg']=data['score_total_esg']
#dd['score total threshold']=data['score_total_threshold']
#dd['inTop18']=data['inTop18']
#dd['Islamic weight']=data['weight_msci_islamic']
#dd['Dividend_NextMonth']=data.iloc[:,402] #dividend_nextMonth

In [18]:
dd.shape

(958, 47)

In [19]:
dd=dd.sort_values('Mkt Cap mn USD',ascending=False)

dd.reset_index(drop=True,inplace=True)

In [20]:
dd.shape

(958, 47)

In [21]:
dd.head()

,Tick | Curr,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Mkt Cap mn USD,Stock Price,Market,Currency,...,Ticker Symbol,Exchange Ticker,Country of HQ,Market adj,Sector Name,Subsector Name,Sub Industry,Dividend Ex Date,Avg_Volume_Daily_USD,Ideal Portfolio
0,RELIANCE | INR,RELIANCE,Reliance Industries,"Oil, Gas & Consumable Fuels",Energy,Reliance Industries Limited is engaged in the ...,201388.525036,1265.10,India,INR,...,RELIANCE,RELIANCE,India,India,Manufacturing,Petroleum and Coal Products Manufacturing,Petroleum Refineries,NaN,184.308959,NaN
1,TCS | INR,TCS,Tata Consultancy Services,IT Services,Information Technology,Tata Consultancy Services Limited (TCS) is an ...,175029.127147,4112.40,India,INR,...,TCS,TCS,India,India,"Professional, Scientific, and Technical Services","Professional, Scientific, and Technical Services",Computer Systems Design and Related Services,NaN,110.555549,NaN
2,HDFCBANK | INR,HDFCBANK,HDFC Bank,Banks,Financials,HDFC Bank Limited (the Bank) is a private bank...,152875.191446,1698.75,India,INR,...,HDFCBANK,HDFCBANK,India,India,Finance and Insurance,Credit Intermediation and Related Activities,Commercial Banking,NaN,218.227445,NaN
3,BHARTIARTL | INR,BHARTIARTL,Bharti Airtel,Wireless Telecommunication Services,Communication Services,Bharti Airtel Limited is a telecommunication c...,114481.325850,1626.30,India,INR,...,BHARTIARTL,BHARTIARTL,India,India,Information,Telecommunications,Wired and Wireless Telecommunications Carriers...,NaN,78.857452,0.033333
4,ICICIBANK | INR,ICICIBANK,Icici Bank,Banks,Financials,ICICI Bank Limited (the Bank) is an India-base...,104063.527329,1252.80,India,INR,...,ICICIBANK,ICICIBANK,India,India,Finance and Insurance,Credit Intermediation and Related Activities,Commercial Banking,NaN,143.172378,NaN


In [22]:
dd['Cap_Weighting']=0

In [23]:
for i in range (0,len(dd)):
    
    if dd['Mkt Cap mn USD'].iloc[i] >= dd['Mkt Cap mn USD'].iloc[99]:
        
        dd['Cap_Weighting'].iloc[i] = 'Large Cap'
        
    elif dd['Mkt Cap mn USD'].iloc[i] >= dd['Mkt Cap mn USD'].iloc[249]:
        
        dd['Cap_Weighting'].iloc[i] = 'Mid Cap'
        
    elif dd['Mkt Cap mn USD'].iloc[i] >= dd['Mkt Cap mn USD'].iloc[499]:
        
        dd['Cap_Weighting'].iloc[i] = 'Small Cap'
        
    else:
        
        dd['Cap_Weighting'].iloc[i] = 'Micro Cap'
    

C:\Users\Vedant Wanchoo\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [24]:
dd.shape

(958, 48)

In [25]:
dd.head()

,Tick | Curr,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Mkt Cap mn USD,Stock Price,Market,Currency,...,Exchange Ticker,Country of HQ,Market adj,Sector Name,Subsector Name,Sub Industry,Dividend Ex Date,Avg_Volume_Daily_USD,Ideal Portfolio,Cap_Weighting
0,RELIANCE | INR,RELIANCE,Reliance Industries,"Oil, Gas & Consumable Fuels",Energy,Reliance Industries Limited is engaged in the ...,201388.525036,1265.10,India,INR,...,RELIANCE,India,India,Manufacturing,Petroleum and Coal Products Manufacturing,Petroleum Refineries,NaN,184.308959,NaN,Large Cap
1,TCS | INR,TCS,Tata Consultancy Services,IT Services,Information Technology,Tata Consultancy Services Limited (TCS) is an ...,175029.127147,4112.40,India,INR,...,TCS,India,India,"Professional, Scientific, and Technical Services","Professional, Scientific, and Technical Services",Computer Systems Design and Related Services,NaN,110.555549,NaN,Large Cap
2,HDFCBANK | INR,HDFCBANK,HDFC Bank,Banks,Financials,HDFC Bank Limited (the Bank) is a private bank...,152875.191446,1698.75,India,INR,...,HDFCBANK,India,India,Finance and Insurance,Credit Intermediation and Related Activities,Commercial Banking,NaN,218.227445,NaN,Large Cap
3,BHARTIARTL | INR,BHARTIARTL,Bharti Airtel,Wireless Telecommunication Services,Communication Services,Bharti Airtel Limited is a telecommunication c...,114481.325850,1626.30,India,INR,...,BHARTIARTL,India,India,Information,Telecommunications,Wired and Wireless Telecommunications Carriers...,NaN,78.857452,0.033333,Large Cap
4,ICICIBANK | INR,ICICIBANK,Icici Bank,Banks,Financials,ICICI Bank Limited (the Bank) is an India-base...,104063.527329,1252.80,India,INR,...,ICICIBANK,India,India,Finance and Insurance,Credit Intermediation and Related Activities,Commercial Banking,NaN,143.172378,NaN,Large Cap


In [26]:
#checking if NAs are getting identified

dd111=dd.dropna(subset=['IB Ticker'])
dd111.shape

(958, 48)

In [27]:
#checking if NAs are getting identified

l=0
a=np.nan
for i in range (0,len(dd)):
    if dd['IB Ticker'].iloc[i] is a:
        l=l+1
l

0

In [28]:
l+len(dd111)==len(dd)

True

In [29]:
l=0
a=np.nan
for i in range (0,len(dd)):
    if dd['IB Ticker'].iloc[i] is a:
        l=l+1

In [30]:
l

0

In [31]:
a= np.nan
for i in range(0,len(dd)):
    if dd['IB Ticker'].iloc[i] is a:
        dd['IB Ticker'].iloc[i]=dd['Exchange Ticker'].iloc[i]


In [32]:
a= np.nan
for i in range(0,len(dd)):
    if dd['IB Ticker'].iloc[i] is a:
        dd['IB Ticker'].iloc[i]=dd['Ticker'].iloc[i]

In [33]:
a= np.nan
for i in range(0,len(dd)):
    if dd['IB Ticker'].iloc[i] is a:
        dd['IB Ticker'].iloc[i]=dd['Ticker Symbol'].iloc[i]

In [34]:
a= np.nan
for i in range(0,len(dd)):
    if dd['IB Ticker'].iloc[i] is a:
        dd['IB Ticker'].iloc[i]=dd['RIC'].iloc[i].split('.')[0]

In [35]:
l=0
a=np.nan
for i in range (0,len(dd)):
    if dd['IB Ticker'].iloc[i] is a:
        l=l+1
l

0

In [36]:
dd.shape

(958, 48)

In [37]:
dd.head()

,Tick | Curr,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Mkt Cap mn USD,Stock Price,Market,Currency,...,Exchange Ticker,Country of HQ,Market adj,Sector Name,Subsector Name,Sub Industry,Dividend Ex Date,Avg_Volume_Daily_USD,Ideal Portfolio,Cap_Weighting
0,RELIANCE | INR,RELIANCE,Reliance Industries,"Oil, Gas & Consumable Fuels",Energy,Reliance Industries Limited is engaged in the ...,201388.525036,1265.10,India,INR,...,RELIANCE,India,India,Manufacturing,Petroleum and Coal Products Manufacturing,Petroleum Refineries,NaN,184.308959,NaN,Large Cap
1,TCS | INR,TCS,Tata Consultancy Services,IT Services,Information Technology,Tata Consultancy Services Limited (TCS) is an ...,175029.127147,4112.40,India,INR,...,TCS,India,India,"Professional, Scientific, and Technical Services","Professional, Scientific, and Technical Services",Computer Systems Design and Related Services,NaN,110.555549,NaN,Large Cap
2,HDFCBANK | INR,HDFCBANK,HDFC Bank,Banks,Financials,HDFC Bank Limited (the Bank) is a private bank...,152875.191446,1698.75,India,INR,...,HDFCBANK,India,India,Finance and Insurance,Credit Intermediation and Related Activities,Commercial Banking,NaN,218.227445,NaN,Large Cap
3,BHARTIARTL | INR,BHARTIARTL,Bharti Airtel,Wireless Telecommunication Services,Communication Services,Bharti Airtel Limited is a telecommunication c...,114481.325850,1626.30,India,INR,...,BHARTIARTL,India,India,Information,Telecommunications,Wired and Wireless Telecommunications Carriers...,NaN,78.857452,0.033333,Large Cap
4,ICICIBANK | INR,ICICIBANK,Icici Bank,Banks,Financials,ICICI Bank Limited (the Bank) is an India-base...,104063.527329,1252.80,India,INR,...,ICICIBANK,India,India,Finance and Insurance,Credit Intermediation and Related Activities,Commercial Banking,NaN,143.172378,NaN,Large Cap


In [38]:
dd2=dd

In [39]:
check=dd2[dd2['kill size']==1]
print(check.shape)

check=dd2[dd2['kill liquidity']==1]
print(check.shape)

check=dd2[(dd2['kill liquidity']==1)+(dd2['kill size']==1)]
print(check.shape)

(0, 48)
(0, 48)
(0, 48)


In [40]:
check.head()

,Tick | Curr,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Mkt Cap mn USD,Stock Price,Market,Currency,...,Exchange Ticker,Country of HQ,Market adj,Sector Name,Subsector Name,Sub Industry,Dividend Ex Date,Avg_Volume_Daily_USD,Ideal Portfolio,Cap_Weighting


In [41]:
NAs=dd2[dd2['Business Description'].isnull()]

In [42]:
NAs.shape

(0, 48)

In [43]:
dd2.shape

(958, 48)

In [44]:
a=dd2.dropna(subset=['Business Description'])

In [45]:
a.shape

(958, 48)

In [46]:
b=a.sort_values(by=['Total Score','Mkt Cap mn USD'],ascending=False)#Total Score

In [47]:
b.head()

,Tick | Curr,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Mkt Cap mn USD,Stock Price,Market,Currency,...,Exchange Ticker,Country of HQ,Market adj,Sector Name,Subsector Name,Sub Industry,Dividend Ex Date,Avg_Volume_Daily_USD,Ideal Portfolio,Cap_Weighting
12,HCLTECH | INR,HCLTECH,HCL Technologies,IT Services,Information Technology,HCL Technologies Limited is a technology compa...,55011.151374,1725.45,India,INR,...,HCLTECH,India,India,"Professional, Scientific, and Technical Services","Professional, Scientific, and Technical Services",Computer Systems Design and Related Services,NaN,83.743463,0.033333,Large Cap
828,KCP | INR,KCP,KCP,Construction Materials,Materials,The KCP Limited is an India-based company that...,317.583207,209.41,India,INR,...,KCP,India,India,Manufacturing,Nonmetallic Mineral Product Manufacturing,Cement Manufacturing,NaN,0.688903,NaN,Micro Cap
196,GLENMARK | INR,GLENMARK,Glenmark Pharmaceuticals,Pharmaceuticals,Health Care,Glenmark Pharmaceuticals Limited is an India-b...,4823.919807,1453.20,India,INR,...,GLENMARK,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.229570,0.033333,Mid Cap
312,NATCOPHARM | INR,NATCOPHARM,Natco Pharma,Pharmaceuticals,Health Care,Natco Pharma Limited is an India-based vertica...,2473.250079,1173.85,India,INR,...,NATCOPHARM,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.321135,0.033333,Small Cap
627,STAR | INR,STAR,Strides Pharma Science,Pharmaceuticals,Health Care,Strides Pharma Science Limited is an India-bas...,743.187888,685.50,India,INR,...,STAR,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,7.741922,NaN,Micro Cap


In [48]:
b.shape

(958, 48)

In [49]:
#b.to_csv('b.csv')

In [50]:
c=b.drop_duplicates(subset=['Business Description'],keep='last')

In [51]:
c.shape

(958, 48)

In [52]:
#b[b['Business Description'].duplicated() == True]

len(b[b.duplicated(['Business Description'], keep=False)])

0

In [53]:
c.reset_index(drop=True,inplace=True)

In [54]:
c.shape

(958, 48)

In [55]:
NAs.shape

(0, 48)

In [56]:
join=pd.concat([c,NAs])

In [57]:
join.shape

(958, 48)

In [58]:
dd2.shape

(958, 48)

In [59]:
dd2=join

In [60]:
dd2.shape

(958, 48)

In [61]:
dd2.head()

,Tick | Curr,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Mkt Cap mn USD,Stock Price,Market,Currency,...,Exchange Ticker,Country of HQ,Market adj,Sector Name,Subsector Name,Sub Industry,Dividend Ex Date,Avg_Volume_Daily_USD,Ideal Portfolio,Cap_Weighting
0,HCLTECH | INR,HCLTECH,HCL Technologies,IT Services,Information Technology,HCL Technologies Limited is a technology compa...,55011.151374,1725.45,India,INR,...,HCLTECH,India,India,"Professional, Scientific, and Technical Services","Professional, Scientific, and Technical Services",Computer Systems Design and Related Services,NaN,83.743463,0.033333,Large Cap
1,KCP | INR,KCP,KCP,Construction Materials,Materials,The KCP Limited is an India-based company that...,317.583207,209.41,India,INR,...,KCP,India,India,Manufacturing,Nonmetallic Mineral Product Manufacturing,Cement Manufacturing,NaN,0.688903,NaN,Micro Cap
2,GLENMARK | INR,GLENMARK,Glenmark Pharmaceuticals,Pharmaceuticals,Health Care,Glenmark Pharmaceuticals Limited is an India-b...,4823.919807,1453.20,India,INR,...,GLENMARK,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.229570,0.033333,Mid Cap
3,NATCOPHARM | INR,NATCOPHARM,Natco Pharma,Pharmaceuticals,Health Care,Natco Pharma Limited is an India-based vertica...,2473.250079,1173.85,India,INR,...,NATCOPHARM,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.321135,0.033333,Small Cap
4,STAR | INR,STAR,Strides Pharma Science,Pharmaceuticals,Health Care,Strides Pharma Science Limited is an India-bas...,743.187888,685.50,India,INR,...,STAR,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,7.741922,NaN,Micro Cap


In [62]:
#Parentheses Removal


dd2['Company Name']=dd2['Company Name'].str.replace("(Holdings)","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("(Holding)","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("(International)","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("(Group)","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("(Industrial)","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("(Registered Share)","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("(Depository Interest)","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("(HKSE Connect Equity)","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("Nil Paid","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("Bonus Rights","",regex=False)


dd2['Company Name']=dd2['Company Name'].str.replace("'A'","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("'B'","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("'D'","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("'E'","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("'H'","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("'I'","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("'L'","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("'N'","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("'O'","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("'P'","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("'R'","",regex=False)
dd2['Company Name']=dd2['Company Name'].str.replace("'V'","",regex=False)

dd2['Business Description']=dd2['Business Description'].str.replace("Ã¢â‚¬â„¢","'",regex=False)
dd2['Business Description']=dd2['Business Description'].str.replace("Ã¢Â€Â™","'",regex=False)
dd2['Business Description']=dd2['Business Description'].str.replace("Ã¢Â€Â¯","'",regex=False)
dd2['Business Description']=dd2['Business Description'].str.replace("dÃ¢Â€ÂdÃ¢Â€Â","'",regex=False)

In [63]:

#dd2.loc[dd2['IB Ticker'] == 'AMARAJABAT', ['IB Ticker']] = 'ARE&M'

''''
#Ticker Cleaning

#dd2.loc[dd2['Company Name'] == 'Sonova N ', ['IB Ticker']] = "SOON"
#dd2.loc[(dd2['Company Name'] == 'Sonova N ') & (dd2['Industry']=='Health Care Equipment & Supplies'), ['IB Ticker']] = "SOON"

dd2.loc[dd2['Company Name'] == 'Sonova N ', ['IB Ticker']] = 'SOON'
dd2.loc[dd2['Company Name'] == 'Frasers Group', ['IB Ticker']] = 'FRAS'

dd2.loc[dd2['Company Name'] == 'Sonova N ', ['Currency']] = 'CHF'
dd2.loc[dd2['Company Name'] == 'Bayer ', ['Currency']] = 'EUR'

dd2.loc[dd2['IB Ticker'] == 'RDSB', ['IB Ticker']] = 'SHEL'
dd2.loc[dd2['IB Ticker'] == 'RDSA', ['IB Ticker']] = 'SHEL'

dd2.loc[dd2['IB Ticker'] == 'ASSA.B', ['Currency']] = 'SEK'
dd2.loc[dd2['IB Ticker'] == 'MBG', ['Currency']] = 'EUR'

dd2.loc[(dd2['IB Ticker'] == 'SAP') & (dd2['Currency']=='CHF'), ['Currency']] = 'EUR'
'''

#dd2.loc[dd2['IB Ticker'] == 'F&N', ['IB Ticker']] = 'F N'

'\'\n#Ticker Cleaning\n\n#dd2.loc[dd2[\'Company Name\'] == \'Sonova N \', [\'IB Ticker\']] = "SOON"\n#dd2.loc[(dd2[\'Company Name\'] == \'Sonova N \') & (dd2[\'Industry\']==\'Health Care Equipment & Supplies\'), [\'IB Ticker\']] = "SOON"\n\ndd2.loc[dd2[\'Company Name\'] == \'Sonova N \', [\'IB Ticker\']] = \'SOON\'\ndd2.loc[dd2[\'Company Name\'] == \'Frasers Group\', [\'IB Ticker\']] = \'FRAS\'\n\ndd2.loc[dd2[\'Company Name\'] == \'Sonova N \', [\'Currency\']] = \'CHF\'\ndd2.loc[dd2[\'Company Name\'] == \'Bayer \', [\'Currency\']] = \'EUR\'\n\ndd2.loc[dd2[\'IB Ticker\'] == \'RDSB\', [\'IB Ticker\']] = \'SHEL\'\ndd2.loc[dd2[\'IB Ticker\'] == \'RDSA\', [\'IB Ticker\']] = \'SHEL\'\n\ndd2.loc[dd2[\'IB Ticker\'] == \'ASSA.B\', [\'Currency\']] = \'SEK\'\ndd2.loc[dd2[\'IB Ticker\'] == \'MBG\', [\'Currency\']] = \'EUR\'\n\ndd2.loc[(dd2[\'IB Ticker\'] == \'SAP\') & (dd2[\'Currency\']==\'CHF\'), [\'Currency\']] = \'EUR\'\n'

In [64]:
dd2.shape

(958, 48)

In [65]:
#dd2=dd2[~dd2['Company Name'].str.contains('Nil Paid')]
#dd2=dd2[~dd2['Company Name'].str.contains('Bonus Rights')]

In [66]:
dd2.shape

(958, 48)

In [67]:
dd2=dd2.sort_values(by=['Total Score','Mkt Cap mn USD'],ascending=False)

In [68]:
dd2.reset_index(drop=True,inplace=True)

In [69]:
dd2.head()

,Tick | Curr,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Mkt Cap mn USD,Stock Price,Market,Currency,...,Exchange Ticker,Country of HQ,Market adj,Sector Name,Subsector Name,Sub Industry,Dividend Ex Date,Avg_Volume_Daily_USD,Ideal Portfolio,Cap_Weighting
0,HCLTECH | INR,HCLTECH,HCL Technologies,IT Services,Information Technology,HCL Technologies Limited is a technology compa...,55011.151374,1725.45,India,INR,...,HCLTECH,India,India,"Professional, Scientific, and Technical Services","Professional, Scientific, and Technical Services",Computer Systems Design and Related Services,NaN,83.743463,0.033333,Large Cap
1,KCP | INR,KCP,KCP,Construction Materials,Materials,The KCP Limited is an India-based company that...,317.583207,209.41,India,INR,...,KCP,India,India,Manufacturing,Nonmetallic Mineral Product Manufacturing,Cement Manufacturing,NaN,0.688903,NaN,Micro Cap
2,GLENMARK | INR,GLENMARK,Glenmark Pharmaceuticals,Pharmaceuticals,Health Care,Glenmark Pharmaceuticals Limited is an India-b...,4823.919807,1453.20,India,INR,...,GLENMARK,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.229570,0.033333,Mid Cap
3,NATCOPHARM | INR,NATCOPHARM,Natco Pharma,Pharmaceuticals,Health Care,Natco Pharma Limited is an India-based vertica...,2473.250079,1173.85,India,INR,...,NATCOPHARM,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.321135,0.033333,Small Cap
4,STAR | INR,STAR,Strides Pharma Science,Pharmaceuticals,Health Care,Strides Pharma Science Limited is an India-bas...,743.187888,685.50,India,INR,...,STAR,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,7.741922,NaN,Micro Cap


In [70]:
dd2.shape

(958, 48)

In [71]:
dd3 = dd2[dd2['Total Score'] !=0] #if you do not want to drop stocks that have a total score = 0 than put a # in the start of this row

In [72]:
dd3.shape

(958, 48)

In [73]:
dd4 = dd3[dd3['Mkt Cap mn USD'] >= 100]

In [74]:
dd4.reset_index(drop=True,inplace=True)

In [75]:
dd4.shape

(958, 48)

In [76]:
dd4.head()

,Tick | Curr,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Mkt Cap mn USD,Stock Price,Market,Currency,...,Exchange Ticker,Country of HQ,Market adj,Sector Name,Subsector Name,Sub Industry,Dividend Ex Date,Avg_Volume_Daily_USD,Ideal Portfolio,Cap_Weighting
0,HCLTECH | INR,HCLTECH,HCL Technologies,IT Services,Information Technology,HCL Technologies Limited is a technology compa...,55011.151374,1725.45,India,INR,...,HCLTECH,India,India,"Professional, Scientific, and Technical Services","Professional, Scientific, and Technical Services",Computer Systems Design and Related Services,NaN,83.743463,0.033333,Large Cap
1,KCP | INR,KCP,KCP,Construction Materials,Materials,The KCP Limited is an India-based company that...,317.583207,209.41,India,INR,...,KCP,India,India,Manufacturing,Nonmetallic Mineral Product Manufacturing,Cement Manufacturing,NaN,0.688903,NaN,Micro Cap
2,GLENMARK | INR,GLENMARK,Glenmark Pharmaceuticals,Pharmaceuticals,Health Care,Glenmark Pharmaceuticals Limited is an India-b...,4823.919807,1453.20,India,INR,...,GLENMARK,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.229570,0.033333,Mid Cap
3,NATCOPHARM | INR,NATCOPHARM,Natco Pharma,Pharmaceuticals,Health Care,Natco Pharma Limited is an India-based vertica...,2473.250079,1173.85,India,INR,...,NATCOPHARM,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.321135,0.033333,Small Cap
4,STAR | INR,STAR,Strides Pharma Science,Pharmaceuticals,Health Care,Strides Pharma Science Limited is an India-bas...,743.187888,685.50,India,INR,...,STAR,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,7.741922,NaN,Micro Cap


In [77]:
dd5 =dd3[dd3['Total Score'] >0]

In [78]:
dd5.shape

(958, 48)

In [79]:
#dd2 = dd2[dd2['Cap_Weighting'] != 'Micro Cap']

In [80]:
#dd2.shape

In [81]:
dd2.reset_index(drop=True,inplace=True)

In [82]:
#

#dd2.to_csv('India_Initial_Universe_2024-06-02.csv')

In [83]:
##

cgsi=pd.read_csv(r'C:\Users\Vedant Wanchoo\Desktop\CGS 2020\India Fund\India Proposal\Portfolio Assembly India\CG_India_Multi_Factor_7_2_2025.csv',skiprows=9)

In [84]:
cgsi.shape

(30, 7)

In [85]:
cgsi.head()

,Name,Ticker,Current Price (Rs.),Avg Buy Price (Rs.),Returns (%),Weightage,Shares
0,Lupin Ltd,LUPIN,2204.60,1809.28,21.84,5.54,4
1,Natco Pharma Ltd,NATCOPHARM,1321.60,981.90,34.59,3.32,4
2,Torrent Pharmaceuticals Ltd,TORNTPHARM,3261.45,3181.95,2.49,4.10,2
3,Aditya Birla Sun Life AMC Ltd,ABSLAMC,702.50,744.75,-5.67,3.53,8
4,HDFC Asset Management Company Ltd,HDFCAMC,3933.35,4322.50,-9.00,2.47,1


In [86]:
cgsi.tail()

,Name,Ticker,Current Price (Rs.),Avg Buy Price (Rs.),Returns (%),Weightage,Shares
25,Alkem Laboratories Ltd,ALKEM,5284.35,5651.80,-6.50,3.32,1
26,Persistent Systems Ltd,PERSISTENT,6225.55,6389.00,-2.55,3.91,1
27,eClerx Services Limited,ECLERX,3203.00,3363.05,-4.75,4.02,2
28,Computer Age Management Services Ltd,CAMS,3669.85,4881.30,-24.81,2.30,1
29,Crompton Greaves Consumer Electricals Ltd,CROMPTON,365.00,365.40,-0.10,3.21,14


In [87]:
cgsi=cgsi[['Ticker','Weightage']]

In [88]:
cgsi.shape

(30, 2)

In [89]:
cgsi['Weightage'] = cgsi['Weightage']/100

In [90]:
cgsi.shape

(30, 2)

In [91]:
cgsi.columns=['IB Ticker','Weight']

In [92]:
cgsi.reset_index(drop=True,inplace=True)

In [93]:
cgsi.shape

(30, 2)

In [94]:
cgsi.head()

,IB Ticker,Weight
0,LUPIN,0.0554
1,NATCOPHARM,0.0332
2,TORNTPHARM,0.0410
3,ABSLAMC,0.0353
4,HDFCAMC,0.0247


In [95]:
dd2.shape

(958, 48)

In [96]:
dd2.head()

,Tick | Curr,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Mkt Cap mn USD,Stock Price,Market,Currency,...,Exchange Ticker,Country of HQ,Market adj,Sector Name,Subsector Name,Sub Industry,Dividend Ex Date,Avg_Volume_Daily_USD,Ideal Portfolio,Cap_Weighting
0,HCLTECH | INR,HCLTECH,HCL Technologies,IT Services,Information Technology,HCL Technologies Limited is a technology compa...,55011.151374,1725.45,India,INR,...,HCLTECH,India,India,"Professional, Scientific, and Technical Services","Professional, Scientific, and Technical Services",Computer Systems Design and Related Services,NaN,83.743463,0.033333,Large Cap
1,KCP | INR,KCP,KCP,Construction Materials,Materials,The KCP Limited is an India-based company that...,317.583207,209.41,India,INR,...,KCP,India,India,Manufacturing,Nonmetallic Mineral Product Manufacturing,Cement Manufacturing,NaN,0.688903,NaN,Micro Cap
2,GLENMARK | INR,GLENMARK,Glenmark Pharmaceuticals,Pharmaceuticals,Health Care,Glenmark Pharmaceuticals Limited is an India-b...,4823.919807,1453.20,India,INR,...,GLENMARK,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.229570,0.033333,Mid Cap
3,NATCOPHARM | INR,NATCOPHARM,Natco Pharma,Pharmaceuticals,Health Care,Natco Pharma Limited is an India-based vertica...,2473.250079,1173.85,India,INR,...,NATCOPHARM,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.321135,0.033333,Small Cap
4,STAR | INR,STAR,Strides Pharma Science,Pharmaceuticals,Health Care,Strides Pharma Science Limited is an India-bas...,743.187888,685.50,India,INR,...,STAR,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,7.741922,NaN,Micro Cap


In [97]:
comb = pd.merge(dd2,cgsi,'left','IB Ticker') # left, outer
comb.shape

(958, 49)

In [98]:
comb.head()

,Tick | Curr,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Mkt Cap mn USD,Stock Price,Market,Currency,...,Country of HQ,Market adj,Sector Name,Subsector Name,Sub Industry,Dividend Ex Date,Avg_Volume_Daily_USD,Ideal Portfolio,Cap_Weighting,Weight
0,HCLTECH | INR,HCLTECH,HCL Technologies,IT Services,Information Technology,HCL Technologies Limited is a technology compa...,55011.151374,1725.45,India,INR,...,India,India,"Professional, Scientific, and Technical Services","Professional, Scientific, and Technical Services",Computer Systems Design and Related Services,NaN,83.743463,0.033333,Large Cap,0.0325
1,KCP | INR,KCP,KCP,Construction Materials,Materials,The KCP Limited is an India-based company that...,317.583207,209.41,India,INR,...,India,India,Manufacturing,Nonmetallic Mineral Product Manufacturing,Cement Manufacturing,NaN,0.688903,NaN,Micro Cap,NaN
2,GLENMARK | INR,GLENMARK,Glenmark Pharmaceuticals,Pharmaceuticals,Health Care,Glenmark Pharmaceuticals Limited is an India-b...,4823.919807,1453.20,India,INR,...,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.229570,0.033333,Mid Cap,0.0290
3,NATCOPHARM | INR,NATCOPHARM,Natco Pharma,Pharmaceuticals,Health Care,Natco Pharma Limited is an India-based vertica...,2473.250079,1173.85,India,INR,...,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.321135,0.033333,Small Cap,0.0332
4,STAR | INR,STAR,Strides Pharma Science,Pharmaceuticals,Health Care,Strides Pharma Science Limited is an India-bas...,743.187888,685.50,India,INR,...,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,7.741922,NaN,Micro Cap,NaN


In [99]:
print(sum(comb['Weight'].dropna()),",",sum(cgsi['Weight']),",",sum(cgsi['Weight'])-sum(comb['Weight'].dropna())) 

0.9988000000000002 , 0.9988000000000002 , 0.0


In [100]:
comb.shape

(958, 49)

In [101]:
comb.reset_index(drop=True,inplace=True)

In [102]:
comb.shape

(958, 49)

In [103]:
comb = comb[~((comb['Cap_Weighting'] == 'Micro Cap') & (comb['Weight'].isna()))]

In [104]:
comb.reset_index(drop=True,inplace=True)

In [105]:
comb.shape

(500, 49)

In [106]:
comb1=comb.copy()

In [107]:
print(sum(comb1['Weight'].dropna()),",",sum(cgsi['Weight']),",",sum(cgsi['Weight'])-sum(comb1['Weight'].dropna())) 

0.9988000000000002 , 0.9988000000000002 , 0.0


In [108]:
comb1.shape

(500, 49)

In [109]:
comb1.head()

,Tick | Curr,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Mkt Cap mn USD,Stock Price,Market,Currency,...,Country of HQ,Market adj,Sector Name,Subsector Name,Sub Industry,Dividend Ex Date,Avg_Volume_Daily_USD,Ideal Portfolio,Cap_Weighting,Weight
0,HCLTECH | INR,HCLTECH,HCL Technologies,IT Services,Information Technology,HCL Technologies Limited is a technology compa...,55011.151374,1725.45,India,INR,...,India,India,"Professional, Scientific, and Technical Services","Professional, Scientific, and Technical Services",Computer Systems Design and Related Services,NaN,83.743463,0.033333,Large Cap,0.0325
1,GLENMARK | INR,GLENMARK,Glenmark Pharmaceuticals,Pharmaceuticals,Health Care,Glenmark Pharmaceuticals Limited is an India-b...,4823.919807,1453.20,India,INR,...,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.229570,0.033333,Mid Cap,0.0290
2,NATCOPHARM | INR,NATCOPHARM,Natco Pharma,Pharmaceuticals,Health Care,Natco Pharma Limited is an India-based vertica...,2473.250079,1173.85,India,INR,...,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.321135,0.033333,Small Cap,0.0332
3,VEDL | INR,VEDL,Vedanta,Metals & Mining,Materials,Vedanta Limited is an India-based natural reso...,20281.599577,441.40,India,INR,...,India,India,Manufacturing,Primary Metal Manufacturing,Alumina and Aluminum Production and Processing,NaN,32.094392,0.033333,Large Cap,NaN
4,APLLTD | INR,APLLTD,Alembic Pharmaceuticals,Pharmaceuticals,Health Care,Alembic Pharmaceuticals Limited is an India-ba...,2127.632350,920.15,India,INR,...,India,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,2.170902,NaN,Small Cap,NaN


In [110]:
comb1['Current_Weight'] = comb1['Weight']

In [111]:
comb1.shape

(500, 50)

In [112]:
comb1.head()

,Tick | Curr,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Mkt Cap mn USD,Stock Price,Market,Currency,...,Market adj,Sector Name,Subsector Name,Sub Industry,Dividend Ex Date,Avg_Volume_Daily_USD,Ideal Portfolio,Cap_Weighting,Weight,Current_Weight
0,HCLTECH | INR,HCLTECH,HCL Technologies,IT Services,Information Technology,HCL Technologies Limited is a technology compa...,55011.151374,1725.45,India,INR,...,India,"Professional, Scientific, and Technical Services","Professional, Scientific, and Technical Services",Computer Systems Design and Related Services,NaN,83.743463,0.033333,Large Cap,0.0325,0.0325
1,GLENMARK | INR,GLENMARK,Glenmark Pharmaceuticals,Pharmaceuticals,Health Care,Glenmark Pharmaceuticals Limited is an India-b...,4823.919807,1453.20,India,INR,...,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.229570,0.033333,Mid Cap,0.0290,0.0290
2,NATCOPHARM | INR,NATCOPHARM,Natco Pharma,Pharmaceuticals,Health Care,Natco Pharma Limited is an India-based vertica...,2473.250079,1173.85,India,INR,...,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,6.321135,0.033333,Small Cap,0.0332,0.0332
3,VEDL | INR,VEDL,Vedanta,Metals & Mining,Materials,Vedanta Limited is an India-based natural reso...,20281.599577,441.40,India,INR,...,India,Manufacturing,Primary Metal Manufacturing,Alumina and Aluminum Production and Processing,NaN,32.094392,0.033333,Large Cap,NaN,NaN
4,APLLTD | INR,APLLTD,Alembic Pharmaceuticals,Pharmaceuticals,Health Care,Alembic Pharmaceuticals Limited is an India-ba...,2127.632350,920.15,India,INR,...,India,Manufacturing,Chemical Manufacturing,Pharmaceutical and Medicine Manufacturing,NaN,2.170902,NaN,Small Cap,NaN,NaN


In [113]:
comb1['Ideal_Weight'] = comb1['Ideal Portfolio']
comb1['Proposed_Weight'] = comb1['Weight']

In [114]:
comb1.shape

(500, 52)

In [115]:
comb1.columns

Index(['Tick | Curr', 'IB Ticker', 'Company Name', 'Industry', 'GICS Sector',
       'Business Description', 'Mkt Cap mn USD', 'Stock Price', 'Market',
       'Currency', 'fx', 'Total Score', 'Value', 'Momentum', 'Quality',
       'Volatility', 'Proprietary', 'ESG Score', 'ESG Combined Score', 'P / S',
       'P / BV', 'P / E', 'P / CF', 'ROE', 'ROA', 'DE', 'DA', 'Dividend Yield',
       'FCF per Share', 'Islam Flag', 'kill size', 'kill liquidity',
       'kill extr Value1', 'kill extr Value2', 'RIC', 'ISIN', 'Ticker',
       'Ticker Symbol', 'Exchange Ticker', 'Country of HQ', 'Market adj',
       'Sector Name', 'Subsector Name', 'Sub Industry', 'Dividend Ex Date',
       'Avg_Volume_Daily_USD', 'Ideal Portfolio', 'Cap_Weighting', 'Weight',
       'Current_Weight', 'Ideal_Weight', 'Proposed_Weight'],
      dtype='object')

In [116]:
comb2=comb1[['IB Ticker', 'Company Name', 'Industry', 'GICS Sector', 'Business Description', 'Total Score','Mkt Cap mn USD','Avg_Volume_Daily_USD',
             'Stock Price', 'Current_Weight', 'Ideal_Weight','Proposed_Weight',
             'Quality', 'Value','Momentum', 'Volatility','Proprietary', 'ESG Combined Score',
             'P / S', 'P / BV', 'P / E', 'P / CF', 'ROE','ROA', 'DE', 'DA', 'Dividend Yield','Islam Flag',
             'RIC', 'ISIN','Cap_Weighting']]

In [117]:
comb2.shape

(500, 31)

In [118]:
##

#comb2=comb2.set_index('IB Ticker')


#comb2.to_csv('India_Port_Universe_2023-09-11.csv')

In [119]:
print(comb2['Ideal_Weight'].sum(),",",comb2['Ideal_Weight'].count())

1.0 , 30


In [120]:
#2024-05-17
##


index=pd.read_excel(r'C:\Users\Vedant Wanchoo\Desktop\CGS 2020\India Fund\India Proposal\Portfolio Assembly India\NIFTY_500_2025-02-07.xlsx')

In [121]:
index.shape

(500, 2)

In [122]:
index.head()

,RIC,NIFTY_500_Weight
0,HDBK.NS,0.073096
1,ICBK.NS,0.049033
2,RELI.NS,0.044738
3,INFY.NS,0.036717
4,ITC.NS,0.024382


In [123]:
index['NIFTY_500_Weight'].sum()

0.999975

In [124]:
comb2.shape

(500, 31)

In [125]:
comb3 = pd.merge(comb2,index,'left','RIC') # left, outer
comb3.shape

(500, 32)

In [126]:
comb3.head()

,IB Ticker,Company Name,Industry,GICS Sector,Business Description,Total Score,Mkt Cap mn USD,Avg_Volume_Daily_USD,Stock Price,Current_Weight,...,ROE,ROA,DE,DA,Dividend Yield,Islam Flag,RIC,ISIN,Cap_Weighting,NIFTY_500_Weight
0,HCLTECH,HCL Technologies,IT Services,Information Technology,HCL Technologies Limited is a technology compa...,43.0,55011.151374,83.743463,1725.45,0.0325,...,NaN,NaN,0.084311,0.057689,0.031296,NaN,HCLT.NS,NaN,Large Cap,0.011005
1,GLENMARK,Glenmark Pharmaceuticals,Pharmaceuticals,Health Care,Glenmark Pharmaceuticals Limited is an India-b...,42.0,4823.919807,6.229570,1453.20,0.0290,...,-0.114593,-0.056382,0.156855,0.085728,0.001720,1.0,GLEN.NS,INE935A01035,Mid Cap,0.001313
2,NATCOPHARM,Natco Pharma,Pharmaceuticals,Health Care,Natco Pharma Limited is an India-based vertica...,42.0,2473.250079,6.321135,1173.85,0.0332,...,0.312583,0.268162,0.063419,0.053748,0.004898,1.0,NATP.NS,INE987B01026,Small Cap,0.000671
3,VEDL,Vedanta,Metals & Mining,Materials,Vedanta Limited is an India-based natural reso...,41.0,20281.599577,32.094392,441.40,NaN,...,NaN,NaN,1.572509,0.381385,0.106857,NaN,VDAN.NS,NaN,Large Cap,0.004085
4,APLLTD,Alembic Pharmaceuticals,Pharmaceuticals,Health Care,Alembic Pharmaceuticals Limited is an India-ba...,41.0,2127.632350,2.170902,920.15,NaN,...,0.138229,0.094456,0.106504,0.079614,0.011955,1.0,ALEM.NS,INE901L01018,Small Cap,0.000323


In [127]:
comb3.insert(12, 'NIFTY_500_Weight',comb3.pop('NIFTY_500_Weight'))

In [128]:
comb3['NIFTY_500_Weight'].sum()

0.9820230000000001

In [129]:
comb3['NIFTY_500_Weight'].fillna(0, inplace=True)

nifty_500_weight_sum = comb3['NIFTY_500_Weight'].sum()

comb3['NIFTY_500_Weight'] = comb3['NIFTY_500_Weight'] / nifty_500_weight_sum

comb3['NIFTY_500_Weight'].replace(0, np.nan, inplace=True)

In [130]:
comb3['NIFTY_500_Weight'].sum()

0.9999999999999999

In [131]:
##

comb4=comb3.set_index('IB Ticker')


comb4.to_csv('India_Port_Universe_SmallCase_2025-02-07.csv')